In [1]:
import numpy as np
import pandas as pd

In [2]:
from env import *

In [3]:
def preprocess(tickpath, filename):
    tickdf = pd.read_csv(os.path.join(tickpath, filename),names=['date','time','ms','lastprice','volume','bid','bidv','ask','askv','opi','tur','contract'])
    tickdf['datestr'] = tickdf['date'].apply(lambda x: '%s-%s-%s'%(str(x)[:4], str(x)[4:6], str(x)[6:8]))
    tickdf['timestr'] = tickdf['time'].apply(lambda x: '%s:%s:%s'%(format(x, '06d')[:-4], format(x, '06d')[-4:-2], format(x, '06d')[-2:]))
    tickdf['timestr'] = tickdf['timestr']+'.'+tickdf['ms'].apply(lambda x: format(x, '03d'))
    tickdf = tickdf.set_index(pd.to_datetime(tickdf['datestr']+' '+tickdf['timestr']))
    tickdf.index.name = 'datetime'
    tickdf.loc[tickdf['bid']<0.9*tickdf['lastprice'], 'bid'] = np.nan
    tickdf.loc[tickdf['ask']<0.9*tickdf['lastprice'], 'ask'] = np.nan
    tickdf[['bid','ask']] = tickdf[['bid','ask']].fillna(method='ffill').fillna(tickdf.iloc[0]['lastprice'])
    tickdf['midprice'] = (tickdf['ask']+tickdf['bid'])/2.0

    tdata = tickdf[['bid','ask','bidv','askv','volume']]
    std = tdata.iloc[0,0]
    tdata.loc[:,'bid'] = tdata['bid']-std
    tdata.loc[:,'ask'] = tdata['ask']-std

    def norm(x):
        return (x-x.mean())/x.std()

    tdata.loc[:,'askv'] = norm(tdata.askv)
    tdata.loc[:,'bidv'] = norm(tdata.bidv)
    tdata.loc[:,'volume'] = norm(tdata.volume)
    return tdata

In [4]:
import os
cwd = '/Data/database/data_zltick/rb'
datas = []
for _,__, files in os.walk(cwd):
    for file in files:
        if int(file[:4])<2020:
            continue
        data = preprocess(cwd, file)
        datas.append(data)

/tmp/ipykernel_50241/3634366836.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdata.loc[:,'bid'] = tdata['bid']-std
/tmp/ipykernel_50241/3634366836.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tdata.loc[:,'ask'] = tdata['ask']-std
/tmp/ipykernel_50241/3634366836.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [5]:
# import matplotlib.pyplot as plt
# fig, ax1 = plt.subplots()

# ax2 = ax1.twinx()
# ax1.plot(tdata.bid[500:800])
# ax2.plot(tdata.volume[500:800], color='red')

# plt.show()

In [6]:
env = BaseMarket(datas=datas,back_length=100,time_limit=600)

In [7]:
tot = 0
env.seed(666)
for i in range(1024):
    done = False
    state = env.reset()
    action = 0
    while not done:
        state, reward, done, _ = env.step(action)
        if state[0] == 1:
            action = 1
    tot += reward
print(tot/1024)

0.201171875


In [8]:
env.seed(666)
tot = 0
for i in range(1024):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1024)

-0.0302734375


In [9]:
import elegantrl.agent as agent
from elegantrl.config import Arguments
env_args = {
    'env_num': 1,
    'env_name': 'executioner',
    'max_step': 600,
    'state_dim': (100+1)*5,
    'action_dim': 2,
    'if_discrete': True,
    'target_return': 0.35
}

In [10]:
env = BaseMarket(datas=datas,back_length=100,time_limit=600)
args = Arguments(agent.AgentD3QN,env=env, env_args=env_args)

In [11]:
from elegantrl.run import train_and_evaluate

In [12]:
args.target_step = args.max_step * 50
args.eval_times = 2 ** 10
args.net_dim = 512
args.batch_size = 1024
args.gamma = 1
args.explore_rate = 0.1
args.seed = 666
args.learning_rate = 2**-15
args.eval_gap = 2 ** 6
train_and_evaluate(args)

| Arguments Remove cwd: ./executioner_D3QN_0
################################################################################
ID     Step    maxR |    avgR   stdR   avgS  stdS |    expR   objC   etc.
0  3.00e+04    0.14 |
0  3.00e+04    0.14 |    0.14    2.5     74   122 |   -0.03   0.04  -0.06


KeyboardInterrupt: 

In [34]:
# age = args.agent().save_or_load_agent(cwd='./executioner_D3QN_0', if_save=False)
from elegantrl.run import *

In [35]:
import torch
torch.set_grad_enabled(False)
# args.init_before_training()
gpu_id = args.learner_gpus

'''init'''
env = build_env(args.env, args.env_func, args.env_args)

age = init_agent(args, gpu_id, env)

In [36]:
age.save_or_load_agent('./executioner_D3QN_other', if_save=False)

In [37]:
# for seed in range(100):
seed = 123
env.seed(seed)
tot = 0
tmp = 0
for i in range(100000):
    done = False
    state = env.reset()
    steps = 0
    # print(state.reshape((-1, 5)))
    while not done:
        steps += 1
        ten_s = torch.as_tensor(state, dtype=torch.float32).unsqueeze(0)
        ten_a = age.act(ten_s)
        # print(ten_a)
        ten_a = ten_a.argmax(dim=1)
        state, reward, done, _ = env.step(ten_a[0].numpy())  # different
        # print(state[-5:])
        # print(reward)
    tot += reward
    tmp += reward
    if i % 1000 == 0:
        print(f'1000 times avg: {tmp/1000}')
        tmp = 0
    # env.episode_data[:300].to_csv('tmp.csv')
    # break
    # print(steps)
print(tot/100000)

1000 times avg: 0.001
1000 times avg: 0.319
1000 times avg: 0.262
1000 times avg: 0.291
1000 times avg: 0.287
1000 times avg: 0.248
1000 times avg: 0.372
1000 times avg: 0.314
1000 times avg: 0.292
1000 times avg: 0.332
1000 times avg: 0.194
1000 times avg: 0.308
1000 times avg: 0.285
1000 times avg: 0.18
1000 times avg: 0.309
1000 times avg: 0.336
1000 times avg: 0.291
1000 times avg: 0.272
1000 times avg: 0.295
1000 times avg: 0.318
1000 times avg: 0.334
1000 times avg: 0.301
1000 times avg: 0.298
1000 times avg: 0.33
1000 times avg: 0.253
1000 times avg: 0.316
1000 times avg: 0.311
1000 times avg: 0.3
1000 times avg: 0.281
1000 times avg: 0.335
1000 times avg: 0.25
1000 times avg: 0.343
1000 times avg: 0.332
1000 times avg: 0.333
1000 times avg: 0.31
1000 times avg: 0.329
1000 times avg: 0.293
1000 times avg: 0.359
1000 times avg: 0.321
1000 times avg: 0.257
1000 times avg: 0.368
1000 times avg: 0.335
1000 times avg: 0.261
1000 times avg: 0.169
1000 times avg: 0.239
1000 times avg: 

In [21]:
for seed in range(100):
    tot = 0
    env.seed(seed)
    for i in range(1000):
        done = False
        state = env.reset()
        action = 0
        while not done:
            state, reward, done, _ = env.step(action)
            if state[0] == 1:
                action = 1
        tot += reward
    print(tot/1000)

0.193
-0.09
-0.106


KeyboardInterrupt: 

In [38]:
tot = 0
env.seed(1)
for i in range(1000):
    done = False
    state = env.reset()
    action = 1
    while not done:
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/1000)

-0.046


In [41]:
tot = 0
env.seed(1)
for i in range(10000):
    done = False
    state = env.reset()
    while not done:
        action = env.action_space.sample()
        state, reward, done, _ = env.step(action)
    tot += reward
print(tot/10000)

-0.0302


In [28]:
tmp = np.load(r'executioner_D3QN_0\replay_0.npz')

In [40]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('tkagg')
plt.plot(data['bid'])
plt.show()

In [35]:
tmp['buf_other']

array([[ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.99,  2.  ],
       [ 0.  ,  0.99,  2.  ],
       [-0.  ,  0.  ,  2.  ]], dtype=float16)

In [5]:
a = [(1,2,3),(3,4,5),(5,6,5),(7,8,5)]
b = list(map(list, zip(*a)))

In [7]:
import torch
b

[[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 5, 5]]